## 2절 플로이드-워셜 최단경로 알고리즘

### 그래프 용어

* 마디 또는 정점(vertex, node)

* 이음선(edge, arc)

* 방향 그래프(directed graph, or digraph)

* 가중치(weight)

* 가중치 포함 그래프(weighted graph)

* 경로(path): 이음선으로 연결된 마디들의 나열. 즉, 하나의 마디에서 다른 마디로 가는 이음선의 연결.

* 단순경로(simple path): 같은 마디를 두 번 지나지 않는 경로

* 순환(cycle): 하나의 마디에서 출발하여 다시 그 마디로 돌아오는 경로

* 순환 그래프(cyclic graph): 순환을 갖는 그래프

* 비순환 그래프 (acyclic graph): 순환을 갖지 않는 그래프

* 경로의 길이(length): 
    * 가중치 포함 그래프의 경우: 경로 상에 있는 가중치의 합
    * 가중치 없는 그래프의 경우: 경로 상에 있는 이음선의 수

### 예제: 가중치 포함 방향그래프

<div align="center"><img src="./images/algo03/algo03-03.png" width="400"/></div>

### 최단경로 문제

* 임의의 하나의 마디에서 다른 임의의 마디로 가는 최단 경로 구하기

* 가중치 포함, 방향성 존중.

* 주의사항: 최단경로는 순환을 포함하지 않아야 함. 즉, 단순경로만 대상으로 삼아도 됨.

#### 예제

* 위 그래프에서 $v_1$에서 $v_3$로 가는 단순경로와 경로 길이:
    * $[v_1, v_2, v_3]$
        * 경로 길이: $1 + 3 = 4$
    * $[v_1, v_4, v_3]$
        * 경로 길이: $1 + 2 = 3$
    * $[v_1, v_2, v_4, v_3]$
        * 경로 길이: $1 + 2 + 2 = 5$        

* 따라서 최단경로와 길이는 다음과 같음.
    * $[v_1, v_4, v_3]$
        * 경로 길이: $1 + 2 = 3$

#### 응용 사례

* 도시 간의 최단 경로

* 다구간 비행기표 여정

* 지도 앱에서 경유 추가

### 최적화 문제

* 하나 이상의 해답 중에서 최적의 값을 갖는 해답을 찾아야 하는 문제

* 최적값: 문제에 따라 최댓값 또는 최솟값을 가리킴.

* 예제: 최단경로 찾기 문제. 
    * 최소 경로길이를 갖는 해답을 찾아야 함.
    * 하나의 마디에서 다른 마디로의 최단경로가 여러 개 있을 수 있음.
    * 그럴 때는 그 중에 하나 선택.

### 최단경로 문제 무작정 알고리즘

* 하나의 마디에서 다른 마디로의 모든 경로의 길이를 계산한 후 그 중에 최소길이 선택.

* 지수보다 나쁜 시간복잡도를 가짐.

#### 무작정 알고리즘 분석

* 가정: 
    * $n$ 개의 마디: $v_1, v_2, ..., v_n$
    * 모든 마디들 사이에 이음선 존재

* $v_1$에서 어떤 마디 $v_n$으로 가는 경로 중 나머지 모든 마디를 한 번씩 꼭 거쳐서 가는 경로들의 수는?
    * $v_1$ 에서 출발하여 처음에 도착할 수 있는 마디의 가지 수는 $(n-2)$ 개
    * 그 중에 하나를 선택하면, 그 다음에 도착할 수 있는 마디의 가지 수는 $(n-3)$개
    * ...
    * 따라서 총 경로의 개수는 다음과 같음:

        $$(n-2)\times(n-3)\times\cdots\times 1= (n-2)!$$

* 이 경로의 수만 보아도 지수보다 훨씬 큼. 따라서 실용성이 전혀 없음.

### 최단경로 알고리즘 동적계획 설계 전략

#### 그래프의 인접행렬

* 마디와 마디를 잇는 이음선과 가중치의 정보를 표현하는 2차원 행렬

* 다음과 같이 정의되는 $n\times n$ 행렬 $W$로 표현할 수 있음.

$$
W[i][j] = 
\begin{cases}
\text{이음선 가중치} & \quad\text{$v_i$ 에서 $v_j$ 로의 이음선이 존재하는 경우} \\
\infty & \quad\text{$v_i$ 에서 $v_j$ 로의 이음선이 존재하지 않는 경우} \\
0 & \quad \text{$i = j$ 인 경우}
\end{cases}
$$

* 예제: 위 예제 그래프의 인접행렬

<div align="center"><img src="./images/algo03/algo03-04.png" width="300"/></div>

#### 최단경로길이 행렬

* 각 마디들 사이의 최단경로의 길이를 담은 2차원 행렬 $D$

* 예제: 위 예제 그래프의 최단경로 길이 행렬

<div align="center"><img src="./images/algo03/algo03-05.png" width="300"/></div>

#### 최단경로길이 행렬 구하기

<div align="center"><img src="./images/algo03/algo03-06.png" width="700"/></div>

#### 동적계획 전략

* 작은 입력사례 살펴보기

* $0 \le k \le n$ 를 만족하는 $k$에 대해 다음을 만족하는 2차원 행렬 $D^{(k)}$ 생성하기

\begin{align*}
D^{(k)}[i][j] &= \text{집합 $\{v_1, v_2, \dots, v_k\}$ 에 속하는 마디만을 통해서} \\
& \quad\,\text{$v_i$ 에서 $v_j$ 로 가는 최단경로의 길이}
\end{align*}

* 다음이 성립함.

\begin{align*}
D^{(0)} &= W \\
D^{(n)} &= D
\end{align*}

* 남은 과제: $0 \le k < n$ 를 만족하는 $k$에 대해, $D^{(k)}$ 로부터 $D^{(k+1)}$ 생성하기.

$$
D^{(0)} \longrightarrow D^{(1)}\longrightarrow D^{(2)}
\longrightarrow \cdots \longrightarrow D^{(n-1)}\longrightarrow D^{(n)}
$$

#### 예제

* 위 예제 그래프에 대해 $D^{(k)}[2][5]$ 계산하기

* $D^{(0)}[2][5] = W[2][5] = \infty$ 
* $D^{(1)}[2][5] = ...$ 


* 참조: [PythonTutor: 동적계획 이항계수 알고리즘](http://pythontutor.com/visualize.html#code=%23%20%EC%9D%B4%EB%B6%84%EA%B2%80%EC%83%89%20%EC%9E%AC%EA%B7%80%0A%0Adef%20bin2%28n,%20k%29%3A%0A%20%20%20%20%23%20n*k%20%EB%AA%A8%EC%96%91%EC%9D%98%20%ED%96%89%EB%A0%AC%20%EC%A4%80%EB%B9%84%ED%95%98%EA%B8%B0.%0A%20%20%20%20%23%20%EB%A6%AC%EC%8A%A4%ED%8A%B8%20%EC%A1%B0%EA%B1%B4%EC%A0%9C%EC%8B%9C%EB%B2%95%20%ED%99%9C%EC%9A%A9%0A%20%20%20%20%0A%20%20%20%20B%20%3D%20%5B%5B0%20for%20_%20in%20range%28k%2B1%29%5D%20for%20_%20in%20range%28n%2B1%29%5D%0A%0A%20%20%20%20for%20i%20in%20range%28n%2B1%29%3A%0A%20%20%20%20%20%20%20%20for%20j%20in%20range%28min%28i,%20k%29%20%2B%201%29%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20if%20j%20%3D%3D%200%20or%20j%20%3D%3D%20i%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20B%5Bi%5D%5Bj%5D%20%3D%201%0A%20%20%20%20%20%20%20%20%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20B%5Bi%5D%5Bj%5D%20%3D%20B%5Bi-1%5D%5Bj-1%5D%20%2B%20B%5Bi-1%5D%5Bj%5D%0A%20%20%20%20%0A%20%20%20%20return%20B%5Bn%5D%5Bk%5D%0A%20%20%20%20%0Abin2%283,%202%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=py3anaconda&rawInputLstJSON=%5B%5D&textReferences=false)